This notebook generates the dataset for the step selection model.

In [1]:
import sys
sys.path.append('../')

import numpy as np
import pandas as pd
import pickle
import os
import glob
import utm
from pathlib import Path
import math
from math import dist
import matplotlib
import matplotlib.pyplot as plt
import cv2
import pandas as pd
from osgeo import gdal
import n2w
gdal.UseExceptions()

import stepselector.preprocessing as pp

/home/blair/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
server_mount = '/home/blair/server/herd_hover'

In [3]:
# If obs_to_process = None, all of the raw track files in the data/raw_tracks folder will be processed
obs_to_process = ['ob015']

# Alternatively, a subset of observations can be processed
#obs_to_process = ['ob015', 'ob027', 'ob036', 'ob053', 'ob074', 'ob088', 'ob090']

# What step length to use?
step_length = 5
offsets = list(range(0, step_length))
dataset = n2w.convert(step_length) + '_meter_steps'

In [4]:
# Define directories
data_folder = os.path.join(server_mount, 'zebra_movement_data')
raw_tracks_directory = os.path.join(data_folder, 'raw_tracks')
dense_tracks_directory = os.path.join(data_folder, 'dense_tracks')
rasters_directory = os.path.join(data_folder, 'rasters')
observed_steps_directory = os.path.join(data_folder, dataset, 'observed')
simulated_steps_directory = os.path.join(data_folder, dataset, 'simulated')
viewshed_save_directory = os.path.join(data_folder, 'visual_fields', dataset)
map_directory = os.path.join(server_mount, '3D_mapping/big_maps')

# Define files
ob_metadata_file = os.path.join(data_folder, 'observation_metadata.csv')
track_metadata_file = os.path.join(data_folder, 'track_metadata.csv')

In [ ]:
# Step 1 - interpolate tracks
pp.interpolate_raw_tracks(raw_tracks_directory = raw_tracks_directory,
                          save_directory = dense_tracks_directory,
                          tolerance = 0.01,
                          obs_to_process = obs_to_process)

In [ ]:
# Step 2 - extract observed steps
pp.extract_observed_steps(step_length = step_length, 
                          offsets = offsets,
                          dense_tracks_directory = dense_tracks_directory,
                          save_directory = observed_steps_directory,
                          rasters_directory = rasters_directory,
                          ob_metadata_file = ob_metadata_file, # change to ob_metadata_file if there's risk of observed steps falling outside mapped area
                          obs_to_process = obs_to_process)

In [ ]:
# Step 3 - simulate fake steps
pp.simulate_fake_steps(n_steps = 20, 
                    observed_steps_directory = observed_steps_directory, 
                    save_directory = simulated_steps_directory, 
                    rasters_directory = rasters_directory, 
                    ob_metadata_file = ob_metadata_file, 
                    obs_to_process = obs_to_process)

In [ ]:
# Step 4 - get observer and step info
pp.get_observer_and_step_info(observed_steps_directory = observed_steps_directory,
                           simulated_steps_directory = simulated_steps_directory,
                           ob_metadata_file = ob_metadata_file,
                           obs_to_process = obs_to_process)

In [ ]:
# Step 5 - calculate zebra observation heights for visibility analyses
pp.calculate_zebra_heights(observed_steps_directory = observed_steps_directory,
                        simulated_steps_directory = simulated_steps_directory,
                        rasters_directory = rasters_directory,
                        ob_metadata_file = ob_metadata_file,
                        obs_to_process = obs_to_process)

In [ ]:
# Step 6 - get step slope
pp.step_slope(observed_steps_directory = observed_steps_directory,
           simulated_steps_directory = simulated_steps_directory,
           rasters_directory = rasters_directory,
           ob_metadata_file = ob_metadata_file,
           obs_to_process = obs_to_process)

In [ ]:
# Step 7 - get social info
pp.get_social_info(observed_steps_directory = observed_steps_directory,
                simulated_steps_directory = simulated_steps_directory,
                raw_tracks_directory = raw_tracks_directory,
                rasters_directory = rasters_directory,
                ob_metadata_file = ob_metadata_file,
                track_metadata_file = track_metadata_file,
                social_radius = 10,
                obs_to_process = obs_to_process)

In [ ]:
# Step 8 - get ground cover & road
pp.get_ground_cover(observed_steps_directory = observed_steps_directory,
              simulated_steps_directory = simulated_steps_directory,
              rasters_directory = rasters_directory,
              obs_to_process = obs_to_process)

In [ ]:
# Step 9 - get track info
pp.get_track_info(observed_steps_directory = observed_steps_directory,
                  simulated_steps_directory = simulated_steps_directory,
                  track_metadata_file = track_metadata_file,
                  obs_to_process = obs_to_process)

In [11]:
# Step 10 - generate viewsheds
pp.preprocess_viewsheds(observed_steps_directory = observed_steps_directory,
                  simulated_steps_directory = simulated_steps_directory,
                  ob_metadata_file = ob_metadata_file,
                  obs_to_process = obs_to_process,
                  map_directory = map_directory,
                  viewshed_save_directory = viewshed_save_directory,
                  keep_rasters = True,
                  radius = 30, # viewshed radius in meters
                  threads = 20) # how many threads to use to generate viewsheds

  0%|          | 0/7 [00:00<?, ?it/s]

observed/ob015_track00_5msteps_0moffset.pkl completed. File processing time:  0:02:56.266970
observed/ob015_track00_5msteps_1moffset.pkl completed. File processing time:  0:02:34.746446
observed/ob015_track00_5msteps_2moffset.pkl completed. File processing time:  0:02:34.720451
observed/ob015_track00_5msteps_3moffset.pkl completed. File processing time:  0:02:35.192546
observed/ob015_track00_5msteps_4moffset.pkl completed. File processing time:  0:02:35.135276
observed/ob015_track01_5msteps_0moffset.pkl completed. File processing time:  0:01:19.787019
observed/ob015_track01_5msteps_1moffset.pkl completed. File processing time:  0:01:19.551506
observed/ob015_track01_5msteps_2moffset.pkl completed. File processing time:  0:01:19.732997
observed/ob015_track01_5msteps_3moffset.pkl completed. File processing time:  0:01:19.589916
observed/ob015_track01_5msteps_4moffset.pkl completed. File processing time:  0:01:18.579986
observed/ob015_track02_5msteps_0moffset.pkl completed. File processing

KeyboardInterrupt: 

In [13]:
# Step 11 - calculate visibility from pre-generated viewsheds
pp.viewshed_visibility(observed_steps_directory = observed_steps_directory,
                       simulated_steps_directory = simulated_steps_directory,
                       viewshed_save_directory = viewshed_save_directory,
                       obs_to_process = obs_to_process)

  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
pp.get_offset(observed_steps_directory = observed_steps_directory,
              simulated_steps_directory = simulated_steps_directory,
              obs_to_process = obs_to_process)

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# Need to add step to add offset column to each file.

# Then push updated preprocessing code and notebook, and make an issue on GitHub for Jake to incorporate 
# necessary changes into dataloader:
# - no "Road" column
# - ground classification is now 1-4, with 4 = road
# - ask about nearest neighbor distance?
# - column for offset - need to make sure context steps are drawn from observed steps file with the 
#       appropriate offset.